<a href="https://colab.research.google.com/github/afaundez/CS74040-2021-fall/blob/homework-1/homework-1/part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PART II:

In this assignment, you will train several language models and will evaluate them on a test corpus. You can discuss in groups, but the homework is to be completed and submitted individually. Two files are provided with this assignment:

1. train.txt
2. test.txt

Each file is a collection of texts, one sentence per line. train.txt contains 10,000 sentences from the NewsCrawl corpus. You will use this corpus to train the language models. The test corpus test.txt is from the same domain and will be used to evaluate the language models that you trained.

In [215]:
train_sentences = open('train.txt').read().strip().split('\n')
print('TRAIN', train_sentences[1])
assert len(train_sentences) == 10000, f'train must have 10000 sentences, got {len(train_sentences)} instead'

TRAIN Man charged over drugs seizure


AssertionError: train must have 10000 sentences, got 100000 instead

In [216]:
test_sentences = open('test.txt').read().strip().split('\n')
print('TEST', test_sentences[2])
assert len(test_sentences) == 100, f'test must have 100 sentences, got {len(test_sentences)} instead'

TEST The road was pitted with tank treads .


## 1.1 PRE-PROCESSING

Prior to training, please complete the following pre-processing steps:


### 1. Pad each sentence in the training and test corpora with start and end symbols (you can use \<s> and \</s>, respectively).

In [217]:
def pad_sentence(sentence, start_pad = '<s>', stop_pad = '</s>'):
    return ' '.join([start_pad] + sentence.split() + [stop_pad])

In [218]:
import re

padded_sentence_pattern = re.compile("^<s> .* </s>$")

padded_train_sentences = [ pad_sentence(sentence) for sentence in train ]
print('TRAIN', padded_train_sentences[1])
assert padded_sentence_pattern.match(padded_train_sentences[1]), f'padded_train_sentences[0] must start with <s> and end with </s>, got {padded_train_sentences[0]} instead'

padded_test_sentences = [pad_sentence(sentence) for sentence in test_sentences]
print('TEST', padded_test_sentences[2])
assert padded_sentence_pattern.match(padded_test_sentences[2]), f'padded_test_sentences[0] must start with <s> and end with </s>, got {padded_test_sentences[0]} instead'

TRAIN <s> Man charged over drugs seizure </s>
TEST <s> The road was pitted with tank treads . </s>


### 2. Lowercase all words in the training and test corpora. Note that the data already has been tokenized (i.e. the punctuation has been split off words).

In [219]:
lowercased_padded_train_sentences = [ sentence.lower() for sentence in padded_train_sentences ]
print('TRAIN', lowercased_padded_train_sentences[1])
assert lowercased_padded_train_sentences[1] == '<s> man charged over drugs seizure </s>', f'lowercased_padded_train_sentences[1] must be "<s> man charged over drugs seizure </s>", got {lowercased_padded_train_sentences[1]} instead'

lowercased_padded_test_sentences = [ sentence.lower() for sentence in padded_test_sentences ]
print('TEST', lowercased_padded_test_sentences[2])
assert lowercased_padded_test_sentences[2] == '<s> the road was pitted with tank treads . </s>', f'lowercased_padded_test_sentences[2] must be "<s> the road was pitted with tank treads . </s>", got {lowercased_padded_test[1]} instead'

TRAIN <s> man charged over drugs seizure </s>
TEST <s> the road was pitted with tank treads . </s>


### 3. Replace all words occurring in the training data once with the token \<unk>. Everyword in the test data not seen in training should be treated as \<unk>.

In [220]:
train_tokens_count_by_token = {}
for sentence in lowercased_padded_train_sentences:
    for word in sentence.split():
        if word not in train_tokens_count_by_token:
            train_tokens_count_by_token[word] = 0
        train_tokens_count_by_token[word] += 1
print(len(train_tokens_count_by_token))
assert train_tokens_count_by_token['<s>'] == 100000, f'start token must be 10000, instead got {train_tokens_count_by_token["<s>"]}'

once_ocurring_words = set(word for word in train_tokens_count_by_token if train_tokens_count_by_token[word] == 1)

assert 'chasnoff' in once_ocurring_words, 'chasnoff must be in once_ocurring_words'

def replace_word(sentence, words, token = '<unk>'):
    return ' '.join([
        token if word in words else word
        for word in sentence.split()
    ])

assert 'chasnoff' in lowercased_padded_train_sentences[2], f'chasnoff must be in {lowercased_padded_train_sentences[2]}'
replaced_lowercased_padded_train_sentences = [ replace_word(sentence, once_ocurring_words) for sentence in lowercased_padded_train_sentences ]
assert 'chasnoff' not in replaced_lowercased_padded_train_sentences[2], f'chasnoff must not be in {replaced_lowercased_padded_train_sentences[2]}'
assert '<unk>' in replaced_lowercased_padded_train_sentences[2], f'<unk> must be in {replaced_lowercased_padded_train_sentences[2]}'

replaced_lowercased_padded_train_vocabulary = set(word for sentence in replaced_lowercased_padded_train_sentences for word in sentence.split())
assert 'leuthard' not in replaced_lowercased_padded_train_vocabulary, 'leuthard is not in train vocabulary'
assert 'septa' not in replaced_lowercased_padded_train_vocabulary, 'septa is not in train vocabulary (replaced with <unk>)'

lowercased_padded_test_vocabulary = set(word for sentence in lowercased_padded_test_sentences for word in sentence.split())
assert 'leuthard' in lowercased_padded_test_vocabulary, 'leuthard is in test vocabulary'
assert 'septa' in lowercased_padded_test_vocabulary, 'septa is in test vocabulary'

test_words_not_in_replaced_lowercased_padded_train_vocabulary = lowercased_padded_test_vocabulary.difference(replaced_lowercased_padded_train_vocabulary)
assert 'leuthard' in test_words_not_in_replaced_lowercased_padded_train_vocabulary, 'leuthard is in test but not in train vocabulary'
assert 'septa' in test_words_not_in_replaced_lowercased_padded_train_vocabulary, 'leuthard is in test but not in train vocabulary'

replaced_lowercased_padded_test_sentences = [ replace_word(sentence, test_words_not_in_replaced_lowercased_padded_train_vocabulary) for sentence in lowercased_padded_test_sentences ]
assert 'leuthard' not in replaced_lowercased_padded_train_vocabulary, 'leuthard is not in train vocabulary'
assert 'septa' not in replaced_lowercased_padded_test_sentences[86], f'septa is not in "{replaced_lowercased_padded_test_sentences[86]}" (replaced with <unk>)'

83045


## 1.2 TRAINING THE MODELS

Please use train.txt to train the following language models:

In [221]:
### 1. A unigram maximum likelihood model.

In [222]:
train_unigrams_count_by_unigram = {} # unigram -> count
for sentence in replaced_lowercased_padded_train_sentences:
    for unigram in sentence.split():
        if unigram not in train_unigrams_count_by_unigram:
            train_unigrams_count_by_unigram[unigram] = 0
        train_unigrams_count_by_unigram[unigram] += 1

print(f'{len(train_unigrams_count_by_unigram)} different unigrams')
train_unigrams_count = sum(train_unigrams_count_by_unigram.values()) # total number of unigrams
print(f'{train_unigrams_count} unigrams instances')

# p(token) = count(token) / total_count
MLE_probabilities_by_unigram = {
    unigram: count / train_unigrams_count
    for unigram, count in train_unigrams_count_by_unigram.items()
} # unigram -> MLE probability

print(sum(MLE_probabilities_by_unigram.values()))

41739 different unigrams
2568210 unigrams instances
1.000000000000187


### 2. A bigram maximum likelihood model.

In [223]:
from itertools import tee

def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [224]:
train_bigrams_count_by_bigram = {} # bigram -> count
for sentence in replaced_lowercased_padded_train_sentences:
    for bigram in pairwise(sentence.split()):
        if bigram not in train_bigrams_count_by_bigram:
            train_bigrams_count_by_bigram[bigram] = 0
        train_bigrams_count_by_bigram[bigram] += 1

print(f'{len(train_bigrams_count_by_bigram)} different bigrams')
train_bigrams_count = sum(train_bigrams_count_by_bigram.values()) # total number of bigrams
print(f'{train_bigrams_count} bigrams instances')

assert train_bigrams_count_by_bigram[('your', '401')] == 1, f'(\'your\', \'401\') bigram count must be 1, instead got {train_bigrams_count_by_bigram[("your", "401")]}'

# p(token | condition) = count(condition, token) / count(condition)
MLE_probabilities_by_token_given_condition = {
    (token, condition): condition_token_count / train_unigrams_count_by_unigram[condition]
    for (condition, token), condition_token_count in train_bigrams_count_by_bigram.items()
} # bigram -> MLE joint probability

# p(condition, token) = p(condition | token) * p(token)
MLE_bigram_joint_probabilities = {
    (condition, token): MLE_probabilities_by_token_given_condition[(condition, token)] * MLE_probabilities_by_unigram[token]
    for (condition, token) in MLE_probabilities_by_token_given_condition.keys()
}

742294 different bigrams
2468210 bigrams instances


In [226]:
import math

assert sum([ value for (token, _), value in MLE_probabilities_by_token_given_condition.items() if token == '<s>' ]) == 0
assert train_unigrams_count_by_unigram['401'] == 15, 'there are 15 instances of token 401 in train'
assert MLE_probabilities_by_unigram['401'] == train_unigrams_count_by_unigram['401'] / train_unigrams_count
assert set(condition for (condition, token) in train_bigrams_count_by_bigram if token == '401' ) == set(['your', 'corp.', 'their', ',', 'of', 'his', ';', 'a', 'a', 'most', 'in', 'of', 'the', 'his', '<s>'])
assert math.isclose(sum([ value * MLE_probabilities_by_unigram[condition] for (token, condition), value in MLE_probabilities_by_token_given_condition.items() if token == '401' ]), MLE_probabilities_by_unigram['401'])

### 3. A bigram model with Add-One smoothing.

### 4. A bigram model with discounting and Katz backoff. Please use a discount constant of 0.5 (see lecture on smoothing).